# <h1> 4-1 로지스틱 회귀 (Logistic Regression)</h1>

* 다중 분류(Multi-class classification) : 타깃 데이터에 2개 이상의 클래스가 포함된 문제 

* 로지스틱 회귀 : 이름은 회귀이지만 분류 모델이며, 선형 방정식을 학습함. 0부터 1까지의 확률을 나타내기 위해 시그모이드 함수(이중 분류) 또는 소프트맥스 함수(다중 분류)를 사용함
  - 시그모이드 함수 : 0에서 1사이의 함수값 $$\phi=\frac{1}{1+e^{-z}}$$

  - 소프트맥스 함수 (Softmax Function) : 다중 분류에서 사용, 지수함수를 이용하여 전체 합이 1이 되도록 만듬. 정규화된 지수 함수 라고도 부름

    1. n개의 z값 (선형방정식의 출력값) $$(z_1, z_2, ..., z_n)$$

    2. 지수함수를 계산하여 모두 더함 $$e.sum = \sum_{k=1}^{n} e^{z_k} = e^{z_1} + e^{z_2} + ... + e^{z_n}$$

    3. 양변을 $e.sum$ 으로 나눔. $s_n$들의 합은 1이 된다. $$s_1 = \frac{e^{z1}}{e.sum}, s_2 = \frac{e^{z2}}{e.sum}, ...$$ 
  
  $$s_1+s_2+ ... +s_n = 1$$


---


<h3> 럭키백의 확률 </h3>


*  맷플롯립, 넘파이, 판다스
*  사이킷런 (sklearn.linear_model 패키지 아래 LogisticRegression 클래스)
* 사이파이 (scipy.special 모듈 아래 expit/softmax 메서드 : 시그모이드/소프트맥스 함수 메서드)

---


1. 로지스틱 회귀는 회귀 모델이 아닌 분류 모델이다. 선형 방정식을 사용하여 계산한 값을 0 ~ 1 사이로 압축하여 확률로 나타낸다.
2. 로지스틱 회귀는 이진 분류에서는 하나의 선형 방정식을 훈련한다. 방정식의 출력값을 시그모이드 함수에 통과시켜 양성 클래스에 대한 확률을 받는다.
3. 다중 분류에서는 클래스 개수만큼 방정식을 훈련한다. 방정식의 출력값들을 소프트맥스 함수에 통과시켜 각 클래스에 대한 확률을 얻는다.


In [ ]:
#############################################################################
#로지스틱 회귀

#럭키백의 확률
#K-최근접 이웃 분류기로 럭키백의 생선을 예측하여 보자

import pandas as pd
fish = pd.read_csv("https://bit.ly/fish_csv_data")      #데이터프레임으로 변환
fish.head()                                             #저장한 데이터 처음 5개 행 확인
print(pd.unique(fish["Species"]))                       #원하는 열에서 고유한 값 추출

#'Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt' : 타깃
#나머지 5개 열 : 입력 데이터

fish_input = fish[["Weight", "Length", "Diagonal", "Height", "Width"]].to_numpy()
# 나머지 5개 열 선택, 넘파이 배열로 변환
print(fish_input[:5])       #처음 5개 행 확인

fish_target = fish["Species"].to_numpy()      #타깃 데이터 만들기

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state = 42)   #훈련세트/테스트세트로 나눈다

from sklearn.preprocessing import StandardScaler  #훈련세트/테스트세트를 표준화 전처리
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

#############################################################################
#K-최근접 이웃 분류기의 확률 예측

from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors = 3)      #이웃의 갯수 3으로 지정
kn.fit(train_scaled, train_target)              #모델 훈련
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))       #훈련세트/테스트세트를 이용하여 평가

#타깃값을 그대로 사이킷런 모델에 전달하면 순서가 자동으로 알파벳 순서로 매겨짐
print(kn.classes_)                              #KNeighborsClassifier에서 정렬된 타깃값 확인

#테스트세트의 처음 5개 샘플의 타깃값을 예측하여보자
print(kn.predict(test_scaled[:5]))

import numpy as np
proba = kn.predict_proba(test_scaled[:5])     #예측한 확률을 출력
print(np.round(proba, decimals=4))            #decimals 매개변수로 소수점 자리 표시갯수 지정
                                              #출력 순서는 classes_ 속성의 순서와 같음

distances, indexes = kn.kneighbors(test_scaled[3:4])  #4번째 샘플의 최근접 이웃의 클래스 확인
print(train_target[indexes])              #확률은 Roach 1/3 , Perch 2/3 : 확률들이 다 똑같다!

#############################################################################
#로지스틱 회귀
#시그모이드 함수(또는 로지스틱 함수)를 이용한다

import numpy as np
import matplotlib.pyplot as plt
z = np.arange(-5,5,0.1)
phi = 1/(1+np.exp(-z))                #np.exp() 함수로 지수 함수 계산
plt.plot(z,phi)
plt.xlabel("Z")
plt.ylabel("Phi")
plt.show()

#############################################################################
#로지스틱 회귀로 이진 분류 수행하기
#간단한 이진분류를 수행하여 보자 (함수값이 0.5보다 작으면 음성 클래스, 크면 양성 클래스)

#불리언 인덱싱 : 넘파이 배열에 True/False 값을 전달하여 행을 선택
char_arr = np.array(["A","B","C","D","E"])
print(char_arr[[True, False, True, False, False]])    #A,C 출력
#이를 이용해 Bream, Smelt 만 골라내어 보자

bream_smelt_indexes = (train_target == "Bream") | (train_target == "Smelt") # or 연산자: |
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)       #로지스틱 회귀 훈련
print(lr.predict(train_bream_smelt[:5]))            #처음 5개 샘플 예측

print(lr.predict_proba(train_bream_smelt[:5]))      #예측 확률 확인
                                                    #1열: 음성클래스 확률, 2열: 양성클래스 확률
print(lr.classes_)          #알파벳 순서, 음성: Bream, 양성: Smelt
print(lr.coef_, lr.intercept_)    #해당 모델의 계수와 절편 확인

#z값 확인 후 직접 확률을 계산하여 보자
decisions = lr.decision_function(train_bream_smelt[:5]) #decision_function 함수에서 z값 출력
print(decisions)

from scipy.special import expit   #scipy.special 모듈의 expit 메서드: 시그모이드 함수
print(expit(decisions))           #z값들을 시그모이드 함수에 입력

#predict_proba 함수 출력 결과의 2번째 열과 같음
# 따라서 decision_function 함수는 양성 클래스에 대한 z값을 출력해줌을 알 수 있음

#############################################################################
#로지스틱 회귀로 다중 분류 수행하기

#LogisticRegression 클래스 : 기본적으로 반복적인 알고리즘을 사용. 계수의 제곱을 규제
#                            max_iter 매개변수로 반복횟수 지정 (기본값은 100)
#                            C 매개변수로 규제의 양 조절 (기본값은 1, 작아질수록 규제 강화)

lr = LogisticRegression(C = 20, max_iter = 1000)
lr.fit(train_scaled, train_target)        # 모델 훈련
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))   #모델 평가

print(lr.predict(test_scaled[:5]))      #테스트세트의 처음 5개 샘플에 대한 예측 출력
proba = lr.predict_proba(test_scaled[:5]) #
print(np.round(proba, decimals = 3))    #예측 확률도 출력 (소수점 아래 셋째 자리까지)

print(lr.classes_)                      #클래스 정보 확인
print(lr.coef_.shape, lr.intercept_.shape)  #선형방정식 확인, (7,5), (7,) 출력
# 7개의 행, 5개의 열 : 다중분류는 클래스마다 z를 하나씩 계산함
# 다중 분류는 소프트맥스 함수를 이용함

#decision_function 메서드로 z1 ~ z7을 구하고 소프트맥스 함수를 사용해 확률로 바꿔보자
decision = lr.decision_function(test_scaled[:5])    #z값들 출력
print(np.round(decision,decimals=2))

from scipy.special import softmax     #소프트맥스 함수 임포트
proba = softmax(decision, axis = 1)   #계산할 축을 지정(axis=1, 각 행(샘플)에 대해 계산)
print(np.round(proba,decimals=3))     #결과 확인

#############################################################################
#로지스틱 회귀로 확률 예측



**===================================================================================================**

# <h1> 4-2 확률적 경사 하강법 (Stochastic Gradient Descent) </h1>

* 확률적 경사 하강법 (Stochastic Gradient Descent): 손실함수 경사를 따라 내려오면서 최적의 모델을 찾는 알고리즘. 샘플을 하나씩 꺼내어 사용한다

* 미니배치 경사 하강법(Minibatch Gradient Descent): 여러 개의 샘플을 꺼내어 사용한다 -> 딥러닝에서 자주 사용

* 배치 경사 하강법(Batch Gradient Descent): 전체 샘플을 꺼내어 사용한다. 그러나 그만큼 컴퓨터 자원을 많이 사용하게 되는 문제가 있다. 

* 에포크 (Epoch): 확률적 경사 하강법에서 전체 샘플을 모두 사용하는 한 번의 반복. 일반적으로 경사 하강법 알고리즘은 수십에서 수백 번의 에포크를 반복함

* 손실함수 (Loss Function): 어떤 문제에서 머신러닝 알고리즘이 얼마나 엉터리인지를 측정하는 기준. 확률적 경사 하강법이 최적화할 대상이다.

* 로지스틱 손실 함수 (Logistic Loss Function): 로지스틱 회귀에서 사용하는 손실함수. 이진분류시 사용. 이진 크로스엔트로피 손실 함수라고도 불림(Binary Cross-Entropy Loss Function)

* 크로스엔트로피 손실 함수 (Cross-Entropy Loss Function): 다중 분류에서 사용하는 손실함수

---


<h3> 점진적 학습 </h3>


*  맷플롯립, 넘파이, 판다스
*  사이킷런 (sklearn.linear_model 모듈 아래 SGDClassifier() 클래스. loss,tol,max_iter 매개변수)

---

1. 대량의 데이터를 사용하는 경우 데이터를 한 번에 컴퓨터 메모리에 읽을 수 없다. 데이터를 조금씩 점진적을 사용하는 방법이 필요하다. 확률적 경사 하강법이 바로 이 문제를 해결하는 핵심 열쇠이다.

2. 손실함수를 우리가 직접 만드는 일은 거의 없다.

3. 적은 에포크는 과소적합, 너무 많은 에포크는 과대적합을 유발할 수 있다.

4. 분류가 아닌 회귀문제에서는 평균 제곱 오차 손실 함수를 사용한다.

In [ ]:
#############################################################################
#확률적 경사 하강법

#점진적인 학습
#데이터가 한번에 준비되는 것이 아닌 조금씩 전달되는 경우

#확률적 경사 하강법
#에포크 : 훈련 세트를 한 번 모두 사용하는 과정
#일반적으로 경사 하강법은 수십, 수백 번 이상 에포크를 수행함

#확률적 경사 하강법: 샘플을 하나 씩 사용해 경사 하강법을 수행
#(Stochastic Gradient Descent)
#미니배치 경사 하강법 : 여러개의 샘플을 사용해 경사 하강법을 수행하는 방식
#(Minibatch Gradient Descent)
#배치 경사 하강법: 한 번 경사로를 따라 이동하기 위해 전체 샘플을 사용하는 방식
#(Batch Gradient Descent)

#손실함수 : 어떤 문제에서 머신러닝 알고리즘이 얼마나 엉터리인지 측정하는 기준

#로지스틱 손실 함수(Logistic Loss Function)
#이진 크로스엔트로피 손실 함수 라고도 불림 (Binary Cross-Entropy Loss Function)
#크로스엔트로피 손실 함수 (Cross-Entropy Loss Function) : 다중 분류에서 사용

#############################################################################
#SGDClassifier
import pandas as pd
fish = pd.read_csv("https://bit.ly/fish_csv_data")    #데이터프레임 만들기
fish_input = fish[["Weight", "Length", "Diagonal", "Height", "Width"]].to_numpy() #5개 특성 입력 데이터로 사용
fish_target = fish["Species"].to_numpy()    #Species 열은 타깃 데이터

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state = 42)       #훈련세트와 테스트세트로 나눔

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)      #훈련세트 표준화 전처리
test_scaled = ss.transform(test_input)        #테스트세트 표준화 전처리 (훈련세트에서 학습한 통계량으로 변환)

from sklearn.linear_model import SGDClassifier    #임포트
sc = SGDClassifier(loss="log", max_iter=10, random_state=42)  
#매개변수 loss="log": 로지스틱 손실함수를 지정, max_iter = 10 : 에포크 횟수를 지정
sc.fit(train_scaled, train_target)                #모델 훈련
print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))         #모델 평가

sc.partial_fit(train_scaled, train_target)        #모델을 이어서 훈련(1 에포크씩 이어서 훈련)
print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))         
#몇 번의 에포크가 적당할까?

#############################################################################
#에포크와 과대/과소적합
#적은 에포크: 과소적합 . 너무 많은 에포크: 과대적합
#조기 종료 (Early Stopping): 과대적합이 시작되기 전에 훈련을 멈추는 것

import numpy as np
sc = SGDClassifier(loss='log', random_state=42)
train_score = []
test_score = []
classes = np.unique(train_target)   #partial_fit 메서드만 사용하려면 전체 클래스의 레이블을 전달해주어야함
                                    #np.unique 함수로 7개 생선의 목록을 만듦

for _ in range(0,300):          # _ 변수: 나중에 사용하지 않고 그냥 버리는 값을 넣어두는 용도
  sc.partial_fit(train_scaled, train_target, classes = classes) #classes 매개변수로 전체 클래스 전달
  train_score.append(sc.score(train_scaled, train_target))
  test_score.append(sc.score(test_scaled, test_target))

import matplotlib.pyplot as plt 
plt.plot(train_score)
plt.plot(test_score)
plt.xlabel("Epoch")
plt.ylabel("Accuarcy")
plt.show()                     #그래프를 통해 비교   
#100번 이후 훈련세트와 테스트세트의 점수가 점점 벌어짐
#반복 횟수를 100번으로 맞추고 모델을 다시 훈련해보자

sc = SGDClassifier(loss="log", max_iter=100, tol=None, random_state=42)
sc.fit(train_scaled, train_target)
print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))
#tol 매개변수: 일정 에포크 동안 성능이 향상되지 않으면 자동으로 멈추는 기능
#              None 으로 설정하여 기능 해제시킴

sc = SGDClassifier(loss="hinge", max_iter=100, tol=None, random_state=42)
sc.fit(train_scaled, train_target)
print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))
#loss 매개변수의 기본값 : "hinge"
#   -> 힌지 손실, 서포트 벡터 머신이라는 머신러닝 알고리즘을 위한 손실함수

#############################################################################
#점진적 학습을 위한 확률적 경사 하강법

 #SGDRegressor() 클래스는 확률적 경사 하강법을 사용한 회귀모델을 만듦

**===================================================================================================**